In [10]:
import logging
import sys
import os

# 스크립트 실행 중 발생하는 로그를 출력
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [18]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor
!pip install pinecone-client
!pip install llama-index llama-index-vector-stores-pinecone

In [20]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import StorageContext
import pinecone



INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: dunzhang/stella_en_1.5B_v5
Load pretrained SentenceTransformer: dunzhang/stella_en_1.5B_v5



You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [28]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone API 설정
os.environ["PINECONE_API_KEY"] = "pcsk_52wVQV_BDBcHcJ86pJCG5ZxB2jK9BEpp45tzGciProEoWt8omizMXKRpnBEjKtdpqsuD4D"

# Pinecone 인스턴스 생성
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "financial2"

# Pinecone 인텍스 생성
pc.create_index(
    name=index_name,
    dimension=1024, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

# Pinecone 인덱스 가져오기
pinecone_index = pc.Index(index_name)


INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/hyottz/anaconda3/envs/HAI/lib/python3.9/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/Users/hyottz/anaconda3/envs/HAI/lib/python3.9/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Installing plugin inference into Pinecone


In [32]:
import pandas as pd
from llama_index.core import Document

# CSV 데이터 로드
df = pd.read_csv("/Users/hyottz/Desktop/24f-houseplan/daiv_houseplan/merged_data.csv")

# Document 객체 생성
documents = []
for _, row in df.iterrows():
    sentence = row["sentence"]
    category = row["category"]
    doc = Document(
    text=sentence,  # 문서 텍스트
    metadata={"category": category}  # 메타데이터
)
    documents.append(doc)



In [41]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex

# Hugging Face 임베딩 모델 로드
embed_model = HuggingFaceEmbedding(model_name="dunzhang/stella_en_1.5B_v5")

# Pinecone VectorStore 설정
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

# LlamaIndex 벡터 저장소 생성
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model, vector_store=vector_store
)



INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: dunzhang/stella_en_1.5B_v5
Load pretrained SentenceTransformer: dunzhang/stella_en_1.5B_v5



You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


Batches: 100%|██████████| 1/1 [00:07<00:00,  7.18s/it]


In [47]:
print(f"VectorStore: {vector_store}")
print(f"Embed Model: {embed_model}")

VectorStore: stores_text=True is_embedding_query=True flat_metadata=False api_key=None index_name=None environment=None namespace=None insert_kwargs={} add_sparse_vector=False text_key='text' batch_size=100 remove_text_from_metadata=False
Embed Model: model_name='dunzhang/stella_en_1.5B_v5' embed_batch_size=10 callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x39cfba100> num_workers=None max_length=512 normalize=True query_instruction=None text_instruction=None cache_folder=None


In [51]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# # Retriever 구성
# retriever = VectorIndexRetriever(
#     index=index,
#     similarity_top_k=5,  # 가장 유사한 문서 5개를 검색
# )

# # Query Engine 조립
# query_engine = RetrieverQueryEngine(
#     retriever=retriever,)

# 쿼리 엔진 생성
query_engine = index.as_query_engine(llm=None)

response = query_engine.query("청년 창업자를 위한 상품은 무엇인가요?")
print(response)



ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [52]:
import os
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.settings import Settings

# Pinecone 인스턴스 생성
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# Pinecone 인덱스 이름 설정
index_name = "financial2"

# Pinecone 인덱스 생성 (없는 경우에만 생성)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Replace with your model dimensions
        metric="cosine",  # Replace with your model metric
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )

# Pinecone 인덱스 가져오기
pinecone_index = pc.Index(index_name)

# Hugging Face 임베딩 모델 로드
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Pinecone VectorStore 설정
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

# CSV 데이터 로드
df = pd.read_csv("/Users/hyottz/Desktop/24f-houseplan/daiv_houseplan/merged_data.csv")

# Document 객체 생성
documents = []
for _, row in df.iterrows():
    sentence = row["sentence"]
    category = row["category"]
    doc = Document(
        text=sentence,  # 문서 텍스트
        metadata={"category": category}  # 메타데이터
    )
    documents.append(doc)

# LlamaIndex 벡터 저장소 생성
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model, vector_store=vector_store
)

# Settings 객체에서 기본 LLM 비활성화
Settings._llm = None

# Hugging Face LLM 설정
hf_llm = HuggingFaceLLM(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 쿼리 엔진 생성
query_engine = index.as_query_engine(llm=hf_llm)

# 질문 수행
response = query_engine.query("청년 창업자를 위한 상품은 무엇인가요?")
print(response)


ModuleNotFoundError: No module named 'llama_index.llms.huggingface'